In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
#Initializing the dataset paths
path ='/content/drive/MyDrive/ALDA_project/Training_dataset/'
testing_path = '/content/drive/MyDrive/ALDA_project/test_set/'
img_path= path+"devset_images/"
label_path = path+ 'devset_images_gt.csv'
test_img_path= testing_path+"testset_images/"
test_label_path = testing_path+ 'testset_images_gt.csv'

In [ ]:
#Formatting the input images
def append_ext(fn):
    return fn+".jpg"
traindf=pd.read_csv(label_path,dtype=str)
traindf.columns=(['id', 'label'])
traindf["id"]=traindf["id"].apply(append_ext)
testdf=pd.read_csv(test_label_path,dtype=str)
testdf.columns=(['id', 'label'])
testdf["id"]=testdf["id"].apply(append_ext)
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.10)
test_datagen = ImageDataGenerator(rescale=1./255.)

In [ ]:
#Creating the dataset generators for train, validation and test
train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=img_path,
x_col="id",
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode='categorical',
target_size=(224,224))
valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=img_path,
x_col="id",
y_col="label",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))
test_generator=datagen.flow_from_dataframe(
dataframe=testdf,
directory=test_img_path,
x_col="id",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 15 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 4738 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 15 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 526 validated image filenames belonging to 2 classes.
Found 1317 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
#Initiazling the image dimensions
img_height = 224
img_width = 224
IMG_SIZE = (img_height, img_width)

In [ ]:
#Defining the pre-trained model

preprocess_input = tf.keras.applications.resnet.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./255., offset= -1)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
image_batch, label_batch = next(iter(train_generator))
feature_batch = base_model(image_batch)
print(feature_batch.shape)
print(label_batch.shape)
base_model.trainable = False

(32, 7, 7, 2048)
(32, 2)


In [ ]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 2048)


In [ ]:
prediction_layer = tf.keras.layers.Dense(2)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 2)


In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
#Extending the pre-trained model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                      

In [ ]:
#Training of the model
initial_epochs = 5
history = model.fit(train_generator,
                    epochs=initial_epochs,
                    validation_data=valid_generator)

Epoch 1/5
149/149 [==============================] - 1065s 7s/step - loss: 0.6768 - accuracy: 0.6112 - val_loss: 0.6403 - val_accuracy: 0.6559
Epoch 2/5
149/149 [==============================] - 879s 6s/step - loss: 0.6686 - accuracy: 0.6178 - val_loss: 0.6361 - val_accuracy: 0.6559
Epoch 3/5
149/149 [==============================] - 877s 6s/step - loss: 0.6600 - accuracy: 0.6241 - val_loss: 0.6304 - val_accuracy: 0.6559
Epoch 4/5
149/149 [==============================] - 881s 6s/step - loss: 0.6561 - accuracy: 0.6199 - val_loss: 0.6368 - val_accuracy: 0.6559
Epoch 5/5
149/149 [==============================] - 886s 6s/step - loss: 0.6540 - accuracy: 0.6188 - val_loss: 0.6238 - val_accuracy: 0.6559


In [ ]:
#Fine-tuning the model
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])
model.summary()

Number of layers in the base model:  175
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
#Fine tuned model training
fine_tune_epochs = 5
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_generator,
                         epochs=total_epochs,
                         initial_epoch=1,
                         validation_data=valid_generator)

Epoch 2/10
149/149 [==============================] - 1767s 12s/step - loss: 0.6108 - accuracy: 0.6543 - val_loss: 0.6196 - val_accuracy: 0.6369
Epoch 3/10
149/149 [==============================] - 1799s 12s/step - loss: 0.6012 - accuracy: 0.6612 - val_loss: 0.5820 - val_accuracy: 0.7072
Epoch 4/10
149/149 [==============================] - 1777s 12s/step - loss: 0.5916 - accuracy: 0.6838 - val_loss: 0.5714 - val_accuracy: 0.7224
Epoch 5/10
149/149 [==============================] - 1796s 12s/step - loss: 0.5886 - accuracy: 0.6834 - val_loss: 0.5770 - val_accuracy: 0.6920
Epoch 6/10
149/149 [==============================] - 1794s 12s/step - loss: 0.5781 - accuracy: 0.6845 - val_loss: 0.5683 - val_accuracy: 0.7300
Epoch 7/10
149/149 [==============================] - 1783s 12s/step - loss: 0.5775 - accuracy: 0.6965 - val_loss: 0.5599 - val_accuracy: 0.7091
Epoch 8/10
149/149 [==============================] - 1777s 12s/step - loss: 0.5725 - accuracy: 0.6973 - val_loss: 0.5629 - val_ac

In [ ]:
#Generating the predictions for test dataset
test_generator.reset()
y_pred = model.predict_generator(test_generator, verbose = True)
y_pred = np.argmax(y_pred, axis=1)
y_true = test_generator.classes 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


42/42 [==============================] - 214s 5s/step


In [ ]:
#Evaluation metrics for the test dataset
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.60      0.61       839
           1       0.33      0.34      0.33       478

    accuracy                           0.51      1317
   macro avg       0.47      0.47      0.47      1317
weighted avg       0.51      0.51      0.51      1317

